# Analyze_CETB_Cubes

Calculate SIR and/or GRD analysis by year for selected subset areas. For example, running melt-onset-dates by year and pixel, or calculating intrapixel stddevs.

Saves analysis data with geolocation information in pickle files.

Makes geotiff maps of various annual and/or average results.


## Load in all the modules needed

In [ ]:
%pylab notebook
# check if a windows machine, it needs special attention
# this extra step will bypass an error from mpl_toolkits.basemap
import os
if os.name == 'nt':
    os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share")
    os.environ["GDAL_DATA"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share", "gdal")
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from netCDF4 import Dataset, num2date
import numpy as np
import pandas as pd
from pathlib import Path
import re
from cetbtools.ease2conv import Ease2Transform
from mpl_toolkits.basemap import Basemap
import glob
from pathlib import Path

# Set parameters to display more than default rows/cols in Data frames

In [ ]:
pd.set_option('display.max_rows', 999)
pd.set_option('display.min_rows', 200)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 10)

In [ ]:
# Define the local machine location of CETB data cubes
# This directory is expected to contain subdirectories in the following hierarchy
# that duplicates the hierarchy on the Google Shared Drive NSIDC-SD-CETB/v1/, 
# for example:
# dataDir/F13_SSMI/N/nc_cubes/cubes_<regionName>
user = 'Joan' #Mariah #MJWindows
if ('Joan' == user):
    dataDir = '/mnt/data3/cetb/nsidc0630_v1/' #jmr machine fringe 
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = Path(Path.home(), 'MODs')
elif ('Mariah' == user):
    dataDir = Path(Path.home(), 'nsidc0630_v1') # Mariah's PC or Mary Jo's Mac
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = Path(Path.home(), 'nsidc0630_v1', 'MODs')
elif ('MJWindows' == user):
    dataDir = Path('Z:/mj On My Mac/nsidc0630_v1') # Mary Jo's Windows machine
    scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')
    outDir = ''
else:
    raise ValueError("unknown user= %s\n" % (user) )
    
%cd $scriptDir
dataDir, outDir, user

In [ ]:
# load the custom functions
from CETB_IO import read_Tb_whole
from CETB_IO import coords
from CETB_algorithms import calc_DAV
# from CETB_IO import find_cube_offset
from CETB_IO import grid_locations_of_subset
from CETB_IO import years_for
from CETB_IO import write_MOD_df_to_geotiff
from CETB_algorithms import DAV_MOD
from CETB_analysis import MOD_array

## Specify inputs

This cell is the place to specify the cube name to analyze ('WesternCA', 'AKYukon', etc)
and the sensor and channels to process.



In [ ]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='AKYukon'#'Laptev' #'WesternCA'  #'GLaIL'  #make this the same syntax as cubefilenames and sub-directory
sat_GRD='F18'   #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sat_SIR= 'F18'
sensor_GRD='SSMIS'  #'AMSRE', 'SSMI', etc.
sensor_SIR='SSMIS'
channel_GRD='37V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
channel_SIR='37V'
alg_GRD='GRD'   #SIR or GRD
alg_SIR='SIR'

# set the sir to grd factor, depends on the channel
if (re.match('^[389]', channel_GRD)):
    sir_2_grd_factor = 8 # assume 3.125 km to 25 km
elif (re.match('^[12]', channel_GRD)):
    sir_2_grd_factor = 4 # assume 6.25 km to 25 km
else:
    raise ValueError("Cannot determine sir_2_grd_factor from channel %s\n" % (channel_GRD) )

cubeType_GRD = channel_GRD + '-' + alg_GRD
cubeType_SIR = channel_SIR + '-' + alg_SIR
  
if ('SSMI' == sensor_GRD) or ('SSMIS' == sensor_GRD):
    provider='CSU' 
    version='v1.*'
elif 'AMSRE' == sensor_GRD:
    provider='RSS'
    version='v1.3'

hemName = 'N'    

# on Joan's machine
#datadir_GRD = dataDir + sat_GRD+'_'+sensor_GRD+'/'+region+'/' 
#datadir_SIR = dataDir + sat_SIR+'_'+sensor_SIR+'/'+region+'/' 
# on MJ's machine
datadir_GRD = "%s/%s_%s/%s/nc_cubes/cubes_%s/" % (
    dataDir, sat_GRD, sensor_GRD, hemName, region )
datadir_SIR = "%s/%s_%s/%s/nc_cubes/cubes_%s/" % (
    dataDir, sat_SIR, sensor_SIR, hemName, region )

# prefix filepath
prefix_GRD = 'CETB.cubefile.'+region+'.'+sat_GRD+'_'+sensor_GRD+'-'+channel_GRD+'-'+alg_GRD+'-'+provider+'-'+version
prefix_SIR = 'CETB.cubefile.'+region+'.'+sat_SIR+'_'+sensor_SIR+'-'+channel_SIR+'-'+alg_SIR+'-'+provider+'-'+version

Years=years_for(sat_GRD)
#might want to truncate Years to subset if very slow during testing
#if we give it more years than available what do we want it to do? 
#warn me but return what it finds

# Truncate Years here for speed 
subYears = Years[0:] 
#subYears = Years



In [ ]:
subYears

## Specify the geographic bounds of the subset area inside the cube to process

Also set the 'Site' name for identifying output files.

In [ ]:
# SPECIFY latitude and longitude in decimal degrees, need to choose lat/lon corners so that we will load
# in a rectangle of pixels within the corners of these coordinates
# These areaname values will be used in output filenames, so they should be relatively short, and should
# not include spaces or other punctuation
areaname='barrow' 

if ('vatna' == areaname):
    lat_start=63.75  
    lat_end=64.88    
    lon_start=-20 
    lon_end=-15  
    #Enter a site name for titles of plots
    Site='Vatnajokull, Iceland'
elif 'hunza' == areaname:
    lat_start=35.9  
    lat_end=37.1   
    lon_start=74 
    lon_end=76 
    #Enter a site name for titles of plots
    Site='Hunza Basin'
elif 'gsl' == areaname:
    lat_start=59.00  
    lat_end=67.00   
    lon_start=-119.00 
    lon_end=-107.00
    #Enter a site name for titles of plots
    Site='Great Slave Lake, Canada'
elif 'bathurst_range' == areaname:
    lat_start=58.00  
    lat_end=69.00   
    lon_start=-125.00 
    lon_end=-106.00
    #Enter a site name for titles of plots
    Site='Bathurst Caribou Range, NWT'
elif 'sz' == areaname:
    lat_start=77.00  
    lat_end=81.00   
    lon_start=89.00 
    lon_end=108.00
    #Enter a site name for titles of plots
    Site='Severnaya Zemlya, Russia'
elif 'barrow' == areaname:
    lat_start=69.50  
    lat_end=71.50    
    lon_start=-158 
    lon_end=-152  
    #Enter a site name for titles of plots
    Site='Barrow/Utkiagvik, AK'
elif ('kuparuk' == areaname):
    lat_start=68.50  
    lat_end=70.50    
    lon_start=-151 
    lon_end=-148  
    #Enter a site name for titles of plots
    Site='Kuparuk Basin, AK'
else: 
    raise ValueError("Unknown area name=%s" % (areaname) )

In [ ]:
# MJB: I think this cell is obsolete, since the output is never used 
# get cube offset for finding row/col
# function is region specific
# find_cube_offset(region, cubeDir=datadir_SIR, cubeType=cubeType_SIR, verbose=False)

In [ ]:
datadir_GRD, prefix_GRD

In [ ]:
# get the GRD pixel IDs for the lat/lon rectangle chosen
# and then calculate the corrsponding SIR pixel row/col numbers
rows_cols_GRD=coords(datadir_GRD, prefix_GRD, lat_start, lat_end, lon_start, lon_end)
rows_cols_env = tuple(np.array(rows_cols_GRD) * sir_2_grd_factor)
print(rows_cols_GRD)
print(rows_cols_env)

In [ ]:
# load GRD Tb data
data_GRD = read_Tb_whole(datadir_GRD, prefix_GRD, subYears,
                         rows_cols_GRD[0], rows_cols_GRD[1], rows_cols_GRD[2], rows_cols_GRD[3])

# load in SIR TB data
data_SIR = read_Tb_whole(datadir_SIR, prefix_SIR, subYears,
                         rows_cols_env[0], rows_cols_env[1], rows_cols_env[2], rows_cols_env[3])

# Information passed back from "read_Tb_whole" reader includes:
# CETB_SIR = data_SIR['TB']   # 3-D Tb time-series array of TB
# data_SIR['cal_date']    # 1-D array of dates, these will get passed to later functions
# data_SIR['cal_year']    # 1-D array of years
# data_SIR['cal_month']   # 1-D array of months
# data_SIR['latitude'], data_SIR['longitude'] # 2-D arrays of subset pixel lat/lons
# data_SIR['x'], data_SIR['y'] # 2-D arrays of subset pixel projected x/y
# data_SIR['gpd'] # name of EASE2 projection that the subset was derived from

In [ ]:
# calculate DAV for the Tb data that was imported
DAV_GRD = calc_DAV(data_GRD['TB'])
DAV_SIR = calc_DAV(data_SIR['TB'])

In [ ]:
DAV_GRD.shape

## TODO: Placeholder here to calculate the std dev of the 64 SIR pixels in each GRD pixel
## Will also need to decide how to save this additional information

## Specify the MOD parameters

window : window for MOD algorithm, default is 1, '10' would be 5 days (remember that the CETB data come in 2 measurements per day)

count : number of Tb/DAV exceedances needed to trigger melt-onset-date

DAV and TB thresholds here are from publications:

Johnson et al 2020 AMSRE rSIR Tb >= 249 DAV>=13 and AMSRE GRD Tb>=243 DAV>=14

From Johnson et al 2020 SSMI rSIR and GRD Tb>=247 DAV>=10

DAV_threshold : diurnal amplitude variation in Kelvins default is 10 (per pubs)
Tb_threshold : TB threshold above which melt is possibly triggered

Colorado (Johnson et al 2020) used 5 times in 7 day window
Patagonia (Monahan and Ramage 2010) and Yukon (Semmens et al 2013?) used 3 times in 5 day window

In [ ]:
window = 1   
count = 1  
# These thresholds are good to use for SSM/I
# TODO: this could also be an if block that sets a default for AMSR-E/AMSR2 as well
DAV_threshold = 10
Tb_threshold = 247

# Create data frames of MOD (or potentially other variables/analysis)

In [ ]:
# MOD of the GRD pixel - avg all years
MOD_DOY_array_GRD, MOD_DOY_GRD_df, meltflag_GRD_df = MOD_array(
    datadir_GRD, prefix_GRD, data_GRD, DAV_GRD, rows_cols_GRD, 
    subYears, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_GRD_df

# the meltflag dataframe 

Has an entry for each date (morning and evening) on rows, and for each pixel on columns 

It contains a 1 for any location/date that the melt criteria were met

In [ ]:
meltflag_GRD_df

In [ ]:
# sir MOD array - MOD will be in day of year (DOY) #changeback to Years for all years
MOD_DOY_array, MOD_DOY_df, meltflag_df = MOD_array(
    datadir_SIR, prefix_SIR, data_SIR, DAV_SIR, rows_cols_env, 
    subYears, window, count, DAV_threshold, Tb_threshold)
MOD_DOY_df

# Notes about changes from original notebooks:

The old notebooks used to call MOD_array to get the average MOD for a set of years 
and then call MOD_array_year for a given year of interest.

Now we just call MOD_array for SIR data and GRD data, and get back a data frame with 
MOD columns for each individual year, and one column for the avg MOD for all the years.

This will run much faster, and can be saved and just re-read from a saved file on disk.

Now we are setting up to save these data and the way to read them in another notebook is:

new = pd.read_pickle(MOD_DOY_filename)

# Save the MOD by year data frames for SIR and GRD to pickle files

Also saving geolocation and melt onset flag data frames

In [ ]:
if not os.path.isdir(outDir):
    os.makedirs(outDir)

MODinfo = "MOD_C%1dW%02dT%03dD%02d" % (count, window, Tb_threshold, DAV_threshold)  
meltflaginfo = "meltflag_C%1dW%02dT%03dD%02d" % (count, window, Tb_threshold, DAV_threshold) 

sirMODBasename = "%s/%s.%s.%s.%s.%s.%s" % (
    outDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, MODinfo)
grdMODBasename = "%s/%s.%s.%s.%s.%s.%s" % (
    outDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, MODinfo)

sirmeltflagBasename = "%s/%s.%s.%s.%s.%s.%s" % (
    outDir, data_SIR['gpd'], region, areaname, sat_SIR, channel_SIR, meltflaginfo)
grdmeltflagBasename = "%s/%s.%s.%s.%s.%s.%s" % (
    outDir, data_GRD['gpd'], region, areaname, sat_GRD, channel_GRD, meltflaginfo)

filename = "%s.pkl" % (sirMODBasename)
MOD_DOY_df.to_pickle(filename)
print("MOD_DOY dataframe saved to %s\n" % filename)

filename = "%s.pkl" % (grdMODBasename)
MOD_DOY_GRD_df.to_pickle(filename)
print("MOD_DOY_GRD dataframe saved to %s\n" % filename)

filename = "%s.pkl" % (sirmeltflagBasename)
meltflag_df.to_pickle(filename)
print("meltflag_df dataframe saved to %s\n" % filename)

filename = "%s.pkl" % (grdmeltflagBasename)
meltflag_GRD_df.to_pickle(filename)
print("meltflag_GRD_df dataframe saved to %s\n" % filename)

# Optionally, save the MOD for each year as a geotiff

In [ ]:
outSIR = write_MOD_df_to_geotiff(MOD_DOY_df, data_SIR['gpd'], sirMODBasename, verbose=True)
outGRD = write_MOD_df_to_geotiff(MOD_DOY_GRD_df, data_GRD['gpd'], grdMODBasename, verbose=True)

In [ ]:
# TODO: Other things to potentially save here

#Examples include: saving the DAV, or saving the std dev
    
    